In [5]:
# Set team names
non_bibs_team_name = "Non-bibs FC"
bibs_team_name = "Bibs FC"

team_dict = {
    "n": non_bibs_team_name, "b": bibs_team_name
}

In [ ]:
from moviepy.config import change_settings

change_settings({
    "IMAGEMAGICK_BINARY": r"C:\\Program Files\\ImageMagick-7.1.2-Q16-HDRI\\magick.exe"
})

def mm_ss_to_seconds(time_float):
    """
    Converts time from MM.SS format to total seconds.
    Example: 1.51 → 111 seconds (1 min + 51 sec)
    """
    minutes = int(time_float)
    seconds = int(round((time_float - minutes) * 100))
    return minutes * 60 + seconds


def match_tracker():
    score = {non_bibs_team_name: 0, bibs_team_name: 0}
    player_goals = {non_bibs_team_name: {}, bibs_team_name: {}}
    player_assists = {non_bibs_team_name: {}, bibs_team_name: {}}
    last_scorer = None
    last_team = None

    team, scorer, assist, time = yield  # Prime the generator

    while True:
        if team not in score:
            raise ValueError(f"Team must be '{non_bibs_team_name}' or '{bibs_team_name}'")

        score[team] += 1
        minute = int(time)

        # Track goals
        if scorer not in player_goals[team]:
            player_goals[team][scorer] = []
        player_goals[team][scorer].append(f"{minute}'")

        # Track assists
        if assist:
            if scorer not in player_assists[team]:
                player_assists[team][scorer] = []
            player_assists[team][scorer].append(assist)

        # Mark the most recent scorer
        last_scorer = scorer
        last_team = team

        # Build output string
        output_lines = []
        for t in [non_bibs_team_name, bibs_team_name]:
            output_lines.append(f"{t}: {score[t]}")
            for player in player_goals[t]:
                times = ",".join(player_goals[t][player])
                assists = "".join(f"({a})" for a in player_assists[t].get(player, []))
                line = f"{player} {times} {assists}".strip()
                if player == last_scorer and t == last_team:
                    line = f">>> {line}"  # Highlight most recent
                output_lines.append(line)
            output_lines.append("")  # Blank line between teams

        output_text = "\n".join(output_lines).strip()

        # Wait for next input and yield output
        team, scorer, assist, time = yield output_text

In [ ]:
def match_tracker():
    score = {non_bibs_team_name: 0, bibs_team_name: 0}
    player_goals = {non_bibs_team_name: {}, bibs_team_name: {}}
    player_assists = {non_bibs_team_name: {}, bibs_team_name: {}}

    team, scorer, assist, time = yield  # Prime the generator

    while True:
        if team not in score:
            raise ValueError(f"Team must be '{non_bibs_team_name}' or '{bibs_team_name}'")

        score[team] += 1
        minute = int(time)

        # Track goals
        if scorer not in player_goals[team]:
            player_goals[team][scorer] = []
        player_goals[team][scorer].append(f"{minute}'")

        # Track assists
        if assist:
            if scorer not in player_assists[team]:
                player_assists[team][scorer] = []
            player_assists[team][scorer].append(assist)

        # Build output string
        output_lines = []
        for t in [non_bibs_team_name, bibs_team_name]:
            output_lines.append(f"{t}: {score[t]}")
            for player in player_goals[t]:
                times = ",".join(player_goals[t][player])
                assists = "".join(f"({a})" for a in player_assists[t].get(player, []))
                output_lines.append(f"{player} {times} {assists}".strip())
            output_lines.append("")  # Blank line between teams

        output_text = "\n".join(output_lines).strip()

        # Wait for next input and yield output
        team, scorer, assist, time = yield output_text

In [ ]:
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip, VideoClip

# === Configure your match video ===
video = VideoFileClip("..\July 15, 2025 Game 2 - Copy.mp4")

highlights = [
    {"start": 1.51, "end": 2.05, "text": "Kick-off"},
    {"time":3.55, "text": "Hand ball by Gerald!", "text_duration": 5},
    {"time":7.07, "team":"n", "scored":"Pavlos","assist":"Tom"},
    {"time":8.20, "team":"n", "scored":"Manzan","assist":"Chris"},
    {"time":10.06, "team":"n", "scored":"Tom","assist":"Alex D"},
    {"time":13.11, "team":"b", "scored":"Gerald","assist":"Stu"},
    {"time":14.56, "team":"n", "scored":"Pavlos","assist":"Chris"},
    {"time":16.00, "text": "Hand ball goal not counted", "text_duration": 5},
    {"time":18.47, "team":"b", "scored":"Kevin","assist":"Alex L"},
    {"time":20.59, "team":"n", "scored":"Pavlos"},
    {"time":22.30, "team":"b", "scored":"Gerald","assist":"David J"},
    {"time":24.25, "team":"b", "scored":"David J","assist":"Gerald"},
    {"time":25.41, "team":"b", "scored":"David J","assist":"Gerald"},
    {"time":26.44, "team":"b", "scored":"Gerald","assist":"Alex L"},
    {"time":29.05, "team":"b", "scored":"Gerald","assist":"Yi"},
    {"time":32.45, "team":"b", "scored":"Gerald","assist":"Yi"},
    {"time":34.35, "team":"b", "scored":"Stu","assist":"Kevin"},
    {"time":34.53, "team":"n", "scored":"Pavlos","assist":"Manzan"},
    {"time":35.11, "team":"b", "scored":"David J","assist":"Kevin"},
    {"time":36.54, "team":"b", "scored":"Stu","assist":"David J"},
    {"time":39.46, "text": "Full Time", "text_duration": 5},
]

# === Create clips with score overlay ===
highlight_clips = []
# Create the generator
tracker = match_tracker()
next(tracker) 
for i, h in enumerate(highlights):
    start = mm_ss_to_seconds(h["start"] if "start" in h else h["time"]) - 10
    end = mm_ss_to_seconds(h["end"] if "end" in h else h["time"]) + 5
    clip = video.subclip(start, end)
    
    # Create score text overlay

    if "text" not in h:
        text = tracker.send((team_dict[h["team"]], h["scored"], h["assist"] if "assist" in h else "", mm_ss_to_seconds(h["time"]) // 60
))
        final_scoreboard = text
        text_duration = h["text_duration"] if "text_duration" in h else 5
    else:
        text = h["text"]
        text_duration = h["text_duration"] if "text_duration" in h else clip.duration

    
    txt = TextClip(
        text, fontsize=36, color='white', font="Arial-Bold", bg_color='black'
    ).set_position(("center", "bottom")).set_duration(text_duration).set_start(clip.duration - text_duration)

    # Function to generate timer frame
    def make_timer(t, start_time=start):
        current_time = start_time + int(t)
        minutes = current_time // 60
        seconds = current_time % 60
        txt_clip = TextClip(
            f"{minutes}:{seconds:02d}",
            fontsize=36, color='white', font="Arial-Bold", bg_color='black'
        ).set_position(("left", "top")).set_duration(clip.duration)
        
        return txt_clip.get_frame(t)  # Return actual frame as NumPy array

    # Create dynamic timer clip
    timer_txt = VideoClip(make_timer, duration=clip.duration)

    composite = CompositeVideoClip([clip, txt, timer_txt])
    highlight_clips.append(composite)

# === Step 1: Count goals and assists ===
from collections import defaultdict

goals = defaultdict(int)
assists = defaultdict(int)

for h in highlights:
    if "scored" in h:
        goals[h["scored"]] += 1
    if "assist" in h:
        assists[h["assist"]] += 1

# === Step 2: Build summary text ===

summary_lines = ["  Player Summary", ""]

# Combine goals and assists into a sortable list
player_stats = []
for player in set(goals.keys()) | set(assists.keys()):
    g = goals[player]
    a = assists[player]
    player_stats.append((g, a, player))  # Sort by goals, then assists

# Sort descending by goals, then assists
player_stats.sort(reverse=True)

# Format summary lines
for g, a, player in player_stats:
    summary_lines.append(f"{player}: {g} goal{'s' if g != 1 else ''}, {a} assist{'s' if a != 1 else ''}")

summary_text = "\n".join(summary_lines)

# === Step 3: Create summary clip ===
summary_clip = TextClip(
    summary_text, fontsize=36, color='white', font="Arial", bg_color='black', size=video.size
).set_duration(15).set_position("center")

scoreboard_clip = TextClip(
    final_scoreboard.replace(">>>", ""), fontsize=36, color='white', font="Arial", bg_color='black', size=video.size
).set_duration(10).set_position("center")

    
# === Concatenate all highlight clips ===
from moviepy.editor import concatenate_videoclips

final_highlights = concatenate_videoclips(highlight_clips + [scoreboard_clip, summary_clip])
final_highlights.write_videofile("match1_highlights2.mp4", codec="libx264", fps=25)
 

Moviepy - Building video match1_highlights2.mp4.
MoviePy - Writing audio in match1_highlights2TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video match1_highlights2.mp4



Moviepy - Done !
Moviepy - video ready match1_highlights2.mp4
